In [3]:
import callbacks
JHU_RECENT, JHU_TIME, CSBS, DATE_MAPPER =callbacks.serve_data(True)

In [12]:
JHU_TIME[(JHU_TIME['Date'] == DATE_MAPPER.iloc[0]['Date']) & (JHU_TIME['confirmed'] >0)].groupby('country').sum()

,lat,lon,county,confirmed,deaths
country,,,,,
China,743.8627,2723.0646,0.0,548,17
Japan,36.0000,138.0000,0.0,2,0
"Korea, South",36.0000,128.0000,0.0,1,0
Taiwan*,23.7000,121.0000,0.0,1,0
Thailand,15.0000,101.0000,0.0,2,0
US,37.0902,-95.7129,0.0,1,0
